In [1]:
import os
print(os.getcwd()) # 이 코드를 주피터 노트북 셀에 입력하고 실행하여 현재 작업 디렉토리를 확인하세요.

c:\Users\kj\Desktop\addSimulator


In [2]:
#pip install Flask-SQLAlchemy

In [3]:

from flask import Flask, render_template, request, jsonify, send_from_directory, redirect, url_for
from flask_socketio import SocketIO, emit
from flask_sqlalchemy import SQLAlchemy
import numpy as np
import json
from datetime import datetime
import os # os 모듈 임포트

# --- 중요한 변경: 프로젝트 기본 경로를 명시적으로 설정 ---
# 이 경로를 여러분의 'project_lck_250605' 폴더의 실제 절대 경로로 직접 지정해주세요.
# 예: "C:/Users/PHANTOM/Desktop/a9a9a9/project_lck_250605"
BASE_PROJECT_PATH = "C:/Users/kj/Desktop/addSimulator" # <-- 여기에 실제 경로 입력!

# Flask 애플리케이션 초기화
# template_folder와 static_folder 경로를 BASE_PROJECT_PATH를 기반으로 설정
app = Flask(__name__,
            template_folder=os.path.join(BASE_PROJECT_PATH, "templates"),
            static_folder=os.path.join(BASE_PROJECT_PATH, "static"))


# SocketIO 초기화에 필요한 SECRET_KEY 설정
app.config['SECRET_KEY'] = 'your_secret_key_for_socketio'

# --- 데이터베이스 설정 추가 ---
# site.db 파일 경로를 BASE_PROJECT_PATH를 기반으로 설정
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + os.path.join(BASE_PROJECT_PATH, 'site.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app) # SQLAlchemy 객체 초기화


# --- 데이터베이스 모델 정의 (게시글) ---
class Post(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(100), nullable=False)
    content = db.Column(db.Text, nullable=False)
    author = db.Column(db.String(20), nullable=False, default='익명')
    date_posted = db.Column(db.DateTime, nullable=False, default=datetime.utcnow)

    def __repr__(self):
        return f"Post('{self.title}', '{self.date_posted}')"

# --- SocketIO 초기화 ---
socketio = SocketIO(app)

# 팀 이름과 인코딩 매핑 (예시, 실제 데이터에 맞게 수정 필요)
teams = {
    "BNK FEARX": 0, "DN Freecs": 1, "Dplus KIA": 2, "DRX": 3, "Gen.G": 4,
    "Hanwha Life Esports": 5, "KT Rolster": 6, "Nongshim RedForce": 7,
    "BRION": 8, "T1": 9
}

# 더미 함수와 데이터를 추가
class DummyPredictor:
    def predict(self, data):
        return np.random.rand()

gr = DummyPredictor()
x_train = {
    'Team': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    'Feature1': np.random.rand(10),
    'Feature2': np.random.rand(10)
}


# --- 라우트 정의 ---

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/Simulator')
def Simulator():
    return render_template('Simulator.html')

@app.route('/bnk_fearx')
def bnk_fearx():
    return render_template('BNK FEARX.html')

@app.route('/dn_freecs')
def dn_freecs():
    return render_template('DN Freecs.html')

@app.route('/dplus_kia')
def dplus_kia():
    return render_template('Dplus KIA.html')

@app.route('/drx')
def drx():
    return render_template('DRX.html')

@app.route('/gen_g')
def gen_g():
    return render_template('Gen.G.html')

@app.route('/hanwha_life_esports')
def hanwha_life_esports():
    return render_template('Hanwha Life Esports.html')

@app.route('/kt_rolster')
def kt_rolster():
    return render_template('KT Rolster.html')

@app.route('/nongshim_redforce')
def nongshim_redforce():
    return render_template('Nongshim RedForce.html')

@app.route('/oksavingsbank_brion')
def okSavingsBank_brion():
    return render_template('BRION.html')

@app.route('/player_comparison')
def player_comparison():
    return render_template('Player comparison.html')

@app.route('/T1')
def T1():
    return render_template('T1.html')

@app.route('/static/<path:filename>')
def serve_static(filename):
    # static_folder는 이미 Flask 앱 초기화 시 설정되었으므로 바로 사용 가능
    return send_from_directory(app.static_folder, filename)

@app.route('/<team>')
def team_page(team):
    if team in teams:
        return render_template(f"{team}.html", team_name=team)  # 팀 이름 전달
    else:
        return "팀 페이지를 찾을 수 없습니다.", 404

import json

json_data = '{"execution_count": null, "data": "some_value"}'
python_data = json.loads(json_data)

print(python_data)  # {"execution_count": None, "data": "some_value"}


@app.route('/predict', methods=['POST'])
def predict():
    data = request.json  # JavaScript에서 받은 JSON 데이터
    print("데이터확인>> ",data)
    team_name = data.get('team')  # 받은 팀 이름
    
    if team_name not in teams:
        return jsonify({"error": "잘못된 팀 이름입니다."})

    team_encoded = teams[team_name]  # 팀 이름을 숫자로 변환
    prediction = gr.predict(x_train[x_train['Team'] == teams[team_name]]).mean()*100  # 예측 수행

    print("팀 숫자 변환 값 :", team_encoded)

    return jsonify({
        "team_name": team_name,
        "win_rate": round(prediction, 2)
    })

# --- SocketIO 이벤트 핸들러 ---
@app.route('/chat')
def chat():
    return render_template('chat.html')

@socketio.on('connect')
def test_connect():
    print('Client connected')
    emit('status', {'msg': '새로운 사용자가 입장했습니다.'}, broadcast=True)

@socketio.on('disconnect')
def test_disconnect():
    print('Client disconnected')
    emit('status', {'msg': '사용자가 퇴장했습니다.'}, broadcast=True)

@socketio.on('message')
def handle_message(data):
    print('received message: ' + str(data))
    emit('message', data, broadcast=True)

# --- 게시판 관련 라우트 추가 ---

@app.route('/board')
def board():
    posts = Post.query.order_by(Post.date_posted.desc()).all()
    return render_template('board.html', posts=posts)

@app.route('/board/new', methods=['GET', 'POST'])
def new_post():
    if request.method == 'POST':
        title = request.form['title']
        content = request.form['content']
        author = request.form.get('author', '익명')

        post = Post(title=title, content=content, author=author)
        db.session.add(post)
        db.session.commit()
        return redirect(url_for('board'))
    return render_template('create_post.html')

@app.route('/board/<int:post_id>')
def post_detail(post_id):
    post = Post.query.get_or_404(post_id)
    return render_template('post_detail.html', post=post)

@app.route('/board/<int:post_id>/update', methods=['GET', 'POST'])
def update_post(post_id):
    post = Post.query.get_or_404(post_id)
    if request.method == 'POST':
        post.title = request.form['title']
        post.content = request.form['content']
        post.author = request.form.get('author', '익명')
        db.session.commit()
        return redirect(url_for('post_detail', post_id=post.id))
    return render_template('update_post.html', post=post)

@app.route('/board/<int:post_id>/delete', methods=['POST'])
def delete_post(post_id):
    post = Post.query.get_or_404(post_id)
    db.session.delete(post)
    db.session.commit()
    return redirect(url_for('board'))

# 댓글 구현
class Comment(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    post_id = db.Column(db.Integer, db.ForeignKey('post.id'), nullable=False)  # 게시글 ID와 연결
    content = db.Column(db.Text, nullable=False)
    author = db.Column(db.String(20), nullable=False, default="익명")
    date_posted = db.Column(db.DateTime, nullable=False, default=datetime.utcnow)

    post = db.relationship('Post', backref=db.backref('comments', lazy=True))  # 댓글과 게시글 연결

    def __repr__(self):
        return f"Comment('{self.content}', '{self.date_posted}')"

@app.route('/board/<int:post_id>/comment', methods=['POST'])
def add_comment(post_id):
    post = Post.query.get_or_404(post_id)  # 해당 게시글 확인
    content = request.form['content']  # 댓글 내용 가져오기
    author = request.form.get('author', '익명')  # 작성자 (익명 기본값)

    comment = Comment(post_id=post.id, content=content, author=author)
    db.session.add(comment)
    db.session.commit()

    return redirect(url_for('post_detail', post_id=post.id))

with app.app_context():
    db.create_all()
# --- 애플리케이션 실행 ---
if __name__ == '__main__':
    # Flask 앱 컨텍스트 내에서 데이터베이스 테이블을 생성합니다.
    # 이 부분은 'app.py' 파일이 직접 실행될 때만 동작합니다.
    with app.app_context():
        db.create_all() # 데이터베이스 테이블 생성 (최초 1회만 실행)

    # Socket.IO를 포함하여 Flask 서버를 실행합니다.
    socketio.run(app, debug=True, use_reloader=False, host='192.168.0.23', port=5000, allow_unsafe_werkzeug=True)

Werkzeug appears to be used in a production deployment. Consider switching to a production web server instead.


{'execution_count': None, 'data': 'some_value'}
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://192.168.0.23:5000
Press CTRL+C to quit
192.168.0.23 - - [11/Jun/2025 09:23:16] "GET / HTTP/1.1" 200 -
192.168.0.23 - - [11/Jun/2025 09:23:16] "GET /favicon.ico HTTP/1.1" 404 -
192.168.0.23 - - [11/Jun/2025 09:23:18] "GET /gen_g HTTP/1.1" 200 -
192.168.0.23 - - [11/Jun/2025 09:23:18] "GET /static/images/Dplus%20KIA.png HTTP/1.1" 304 -
192.168.0.23 - - [11/Jun/2025 09:23:18] "GET /static/images/DN%20freecs.png HTTP/1.1" 304 -
192.168.0.23 - - [11/Jun/2025 09:23:18] "GET /static/images/BNK%20FEARX.png HTTP/1.1" 304 -
192.168.0.23 - - [11/Jun/2025 09:23:18] "GET /static/images/DRX.png HTTP/1.1" 304 -
192.168.0.23 - - [11/Jun/2025 09:23:18] "GET /static/player/Kiin.png HTTP/1.1" 304 -
192.168.0.23 - - [11/Jun/2025 09:23:18] "GET /static/images/Gen.G.png HTTP/1.1" 304 -
192.168.0.23 - - [11/Jun/2025 09:23:18] "GET /static/images/Hanwha%20Life%20Esports.png HTTP/1.1" 304 -
192.168.0.23 - - [11/Jun/2025 09:23:18] "GET /static/player/Canyon.png HTTP/1.1" 304 -
192.168.0.23 

데이터확인>>  {'team': 'Gen.G'}
Client connected


192.168.0.23 - - [11/Jun/2025 09:23:29] "GET /socket.io/?EIO=4&transport=websocket&sid=SB04IF-q0GX88h8_AAAA HTTP/1.1" 200 -
192.168.0.23 - - [11/Jun/2025 09:23:29] "GET /gen_g HTTP/1.1" 200 -
192.168.0.23 - - [11/Jun/2025 09:23:29] "GET /static/images/BNK%20FEARX.png HTTP/1.1" 304 -
192.168.0.23 - - [11/Jun/2025 09:23:29] "GET /static/images/DN%20freecs.png HTTP/1.1" 304 -
192.168.0.23 - - [11/Jun/2025 09:23:29] "GET /static/images/Dplus%20KIA.png HTTP/1.1" 304 -
192.168.0.23 - - [11/Jun/2025 09:23:29] "GET /static/images/Hanwha%20Life%20Esports.png HTTP/1.1" 304 -
192.168.0.23 - - [11/Jun/2025 09:23:29] "GET /static/images/Gen.G.png HTTP/1.1" 304 -
192.168.0.23 - - [11/Jun/2025 09:23:29] "GET /static/images/DRX.png HTTP/1.1" 304 -
192.168.0.23 - - [11/Jun/2025 09:23:29] "GET /socket.io/?EIO=4&transport=polling&t=PTSNjZw HTTP/1.1" 200 -
192.168.0.23 - - [11/Jun/2025 09:23:29] "GET /static/player/Kiin.png HTTP/1.1" 304 -
192.168.0.23 - - [11/Jun/2025 09:23:29] "GET /static/player/Canyon

Client disconnected
데이터확인>>  {'team': 'Gen.G'}
Client connected


192.168.0.23 - - [11/Jun/2025 09:23:37] "GET /socket.io/?EIO=4&transport=websocket&sid=Dt72i93-Wgi07qLaAAAC HTTP/1.1" 200 -
192.168.0.23 - - [11/Jun/2025 09:23:37] "GET /bnk_fearx HTTP/1.1" 200 -
192.168.0.23 - - [11/Jun/2025 09:23:37] "GET /static/images/BNK%20FEARX.png HTTP/1.1" 304 -
192.168.0.23 - - [11/Jun/2025 09:23:37] "GET /static/images/DN%20freecs.png HTTP/1.1" 304 -
192.168.0.23 - - [11/Jun/2025 09:23:37] "GET /static/images/Dplus%20KIA.png HTTP/1.1" 304 -
192.168.0.23 - - [11/Jun/2025 09:23:37] "GET /socket.io/?EIO=4&transport=polling&t=PTSNlXY HTTP/1.1" 200 -
192.168.0.23 - - [11/Jun/2025 09:23:37] "GET /static/images/Hanwha%20Life%20Esports.png HTTP/1.1" 304 -
192.168.0.23 - - [11/Jun/2025 09:23:37] "GET /static/images/OKSavingsBank%20BRION.png HTTP/1.1" 304 -
192.168.0.23 - - [11/Jun/2025 09:23:37] "GET /static/player/Raptor.png HTTP/1.1" 200 -
192.168.0.23 - - [11/Jun/2025 09:23:37] "GET /static/player/VicLa.png HTTP/1.1" 200 -
192.168.0.23 - - [11/Jun/2025 09:23:37] "P

Client disconnected
데이터확인>>  {'team': 'BNK FEARX'}
Client connected


192.168.0.23 - - [11/Jun/2025 09:24:02] "GET /socket.io/?EIO=4&transport=websocket&sid=BlPNM9FWYQ66LcwMAAAE HTTP/1.1" 200 -
192.168.0.23 - - [11/Jun/2025 09:24:02] "GET /T1 HTTP/1.1" 200 -
192.168.0.23 - - [11/Jun/2025 09:24:02] "GET /static/images/BNK%20FEARX.png HTTP/1.1" 304 -
192.168.0.23 - - [11/Jun/2025 09:24:02] "GET /static/images/DN%20freecs.png HTTP/1.1" 304 -
192.168.0.23 - - [11/Jun/2025 09:24:02] "GET /static/images/Gen.G.png HTTP/1.1" 304 -
192.168.0.23 - - [11/Jun/2025 09:24:02] "GET /static/images/Hanwha%20Life%20Esports.png HTTP/1.1" 304 -
192.168.0.23 - - [11/Jun/2025 09:24:02] "GET /static/images/Dplus%20KIA.png HTTP/1.1" 304 -
192.168.0.23 - - [11/Jun/2025 09:24:02] "GET /socket.io/?EIO=4&transport=polling&t=PTSNraa HTTP/1.1" 200 -
192.168.0.23 - - [11/Jun/2025 09:24:02] "GET /static/player/DORAN.png HTTP/1.1" 304 -
192.168.0.23 - - [11/Jun/2025 09:24:02] "GET /static/player/Gumayusi.png HTTP/1.1" 304 -
192.168.0.23 - - [11/Jun/2025 09:24:02] "GET /static/player/Ker

Client disconnected
데이터확인>>  {'team': 'T1'}
Client connected


In [ ]:
#pip install flask_sqlalchemy